# Design Thinking using Semantic Kernel

This is a humble implementation in Python of the amazing "Mr Maeda Cozy AI Kitchen", the original notebooks are written in C#: https://github.com/johnmaeda/Mr-Maeda-Cozy-AI-Kitchen/tree/main/DesignThinking.

The idea of running an end-to-end automated Design Thinking process to analyze a problem and generate potential solutions is very appealing to me, and I think it is a great way to introduce people to the power of AI.

### Step 1

Load configuration (don't forget to update it from 1-start-here notebook)

In [16]:
# Import the libraries required for the notebook
from IPython.display import Markdown
import asyncio

# Instatiate the config class
import sys
sys.path.append('..')
from config.notebook_config import *
config = notebook_config()
# Load config from file
config.load_config_from_file()

### Step 2

Initiate semantic kernel.


In [17]:
# Load Semantic Kernel
import semantic_kernel

kernel = semantic_kernel.Kernel()

print("Kernel loaded.")

Kernel loaded.


### Step 3

Fire-up the semantic kernel with the configured OpenAI model.

In [18]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

deployment = config.model
endpoint = config.endpoint
api_key = config.azure_api_key
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))

print(f"Fire-up the kernel with {deployment}")

Fire-up the kernel with gpt-35-turbo


### Step 4

Load the plug-ins to the semantic kernel.

In [19]:
plugins_directory = "../plugins-sk"

# Import the semantic functions
my_functions = kernel.import_semantic_skill_from_directory(plugins_directory, "DesignThinking")

print("Plugins loaded.")

Plugins loaded.


### Step 5

Use one of the plug-ins to generate a sentiment response to a given input (using "empathize" plug-in).

In [20]:
input_str = "1. The car doesn't start \
2. The engine makes a strange noise \
3. The mechanic was very helpful \
4. No one could identify the problem \
5. The GPS is not working properly"

# Create a new task for the coroutine
task = asyncio.create_task(kernel.run_async(my_functions["Empathize"], input_str=input_str))

# Run the event loop until the task completes
empathize_result = await task
print(empathize_result)

[
    { "sentiment": "negative", "summary": "The car doesn't start" },
    { "sentiment": "negative", "summary": "The engine makes a strange noise" },
    { "sentiment": "positive", "summary": "The mechanic was very helpful" },
    { "sentiment": "negative", "summary": "No one could identify the problem" },
    { "sentiment": "negative", "summary": "The GPS is not working properly" }
]


### Step 6

Feed the previous "empathize" output to the next step of Design Thinking, "Define" (using the "define" plug-in).

In [21]:
# Use empathize_result to run the next function
task = asyncio.create_task(kernel.run_async(my_functions["Define"], input_str=empathize_result.result))
define_result = await task

display(Markdown(define_result.result))


| Analysis | Definition | Possible Source |
| :--- | :--- | :--- |
| The car doesn't start | The customer is experiencing difficulty starting their car. | Possible issues with the battery, ignition system, or fuel system. |
| The engine makes a strange noise | The customer hears unusual sounds coming from their car's engine. | Possible issues with the engine components, such as belts, bearings, or valves. |
| The mechanic was very helpful | The customer had a positive experience with a mechanic. | Good customer service and expertise of the mechanic. |
| No one could identify the problem | The customer's car issue could not be diagnosed by any support staff. | Complex or rare problem that requires specialized knowledge or equipment. |
| The GPS is not working properly | The customer is facing issues with the functionality of their car's GPS system. | Software or hardware malfunction in the GPS system. |

### Step 7

Execute a pipeline, with several plug-ins in a row, to generate a full Design Thinking process.

In [55]:
# Input string
input_str = "Customer 1: The car doesn't start \
Customer 2. The engine makes a strange noise \
Customer 3. The mechanic was very helpful \
Customer 4. No one could identify the problem \
Customer 5. The GPS is not working properly"

# Define the pipeline functions
empathize = kernel.skills.get_function("DesignThinking", "Empathize")
define = kernel.skills.get_function("DesignThinking", "Define")
ideate = kernel.skills.get_function("DesignThinking", "Ideate")
prototype = kernel.skills.get_function("DesignThinking", "PrototypeWithPaper")

# Create a new task for the coroutine
task = asyncio.create_task(kernel.run_async(empathize, define, ideate, prototype, input_str=input_str))

# Run the event loop until the task completes
output = await task

display(Markdown(output.result))

1. Implement a battery testing tool to quickly diagnose battery issues in cars that don't start.

Low-resolution paper prototype: Create a simple paper tool that mechanics can use to simulate the battery testing process. The tool can have different sections representing the battery voltage levels and a dial to indicate the result. Mechanics can use this tool to practice diagnosing battery issues and understand how to interpret the results.

2. Provide training and resources to mechanics to improve their knowledge and skills in diagnosing and fixing engine problems.

Low-resolution paper prototype: Design a paper-based training manual that covers various engine problems and their solutions. The manual can include step-by-step instructions, diagrams, and troubleshooting tips. Mechanics can go through the manual and simulate diagnosing and fixing engine problems using the provided information.

3. Develop a customer feedback system to recognize and reward helpful mechanics, encouraging positive customer experiences.

Low-resolution paper prototype: Create a paper feedback form that customers can fill out after their car is repaired. The form can include questions about the mechanic's helpfulness, professionalism, and overall satisfaction. Mechanics can collect and analyze the feedback to understand areas of improvement and identify mechanics who consistently receive positive feedback.

4. Collaborate with other repair shops or experts in the industry to seek assistance in identifying and resolving complex car issues.

Low-resolution paper prototype: Design a paper-based collaboration request form that repair shops can fill out when they need assistance with complex car issues. The form can include details about the problem, the expertise required, and contact information. Repair shops can send these forms to other shops or experts to seek their assistance and expertise.

5. Offer a warranty or guarantee for customers who do not receive a resolution to their car problem, ensuring their satisfaction and providing alternative solutions.

Low-resolution paper prototype: Create a paper warranty certificate that outlines the terms and conditions of the warranty or guarantee. The certificate can include information about the duration of the warranty, what issues are covered, and the process for claiming the warranty. Customers can receive this certificate after their car is repaired, providing them with assurance and alternative solutions if their problem persists.